In [3]:
import pandas as pd

In [4]:
import sqlite3

In [7]:
orders=pd.read_csv("orders - orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [12]:
orders.shape

(10000, 6)

In [11]:
users=pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [13]:
users.shape

(3000, 4)

In [14]:
con=sqlite3.connect("restaurants.db")

In [15]:
with open("restaurants.sql","r") as f:
    sql_script=f.read()
con.executescript(sql_script)

In [16]:
restaurants=pd.read_sql("select * from restaurants",con)

In [17]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [18]:
restaurants.tail()

,restaurant_id,restaurant_name,cuisine,rating
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9
499,500,Restaurant_500,Chinese,3.2


In [20]:
order_merge_user=pd.merge(
    orders  ,users ,
    on="user_id",
    how="left"
)

In [22]:
order_merge_user.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [37]:
restaurant_merge=pd.merge(
    order_merge_user,restaurants,
    on="restaurant_id",
    how="left"
)

In [39]:
restaurant_merge.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [42]:
restaurant_merge = restaurant_merge.drop(
    columns=["restaurant_name_x"],
    errors="ignore"
)


In [43]:
restaurant_merge = restaurant_merge.rename(
    columns={"restaurant_name_y": "restaurant_name"}
)


In [44]:
restaurant_merge.head()

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [48]:
restaurant_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   name             10000 non-null  object 
 6   city             10000 non-null  object 
 7   membership       10000 non-null  object 
 8   restaurant_name  10000 non-null  object 
 9   cuisine          10000 non-null  object 
 10  rating           10000 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 859.5+ KB


In [78]:
restaurant_merge.isna().sum()


order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
name               0
city               0
membership         0
restaurant_name    0
cuisine            0
rating             0
rating_range       0
dtype: int64

In [49]:
restaurant_merge.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

In [47]:
import os
os.listdir()

['.ipynb_checkpoints',
 'data_integration.ipynb',
 'final_food_delivery_dataset.csv',
 'orders - orders.csv',
 'restaurants.db',
 'restaurants.sql',
 'users.json']

1.Order trends over time

In [50]:
restaurant_merge["order_date"] = pd.to_datetime(
    restaurant_merge["order_date"],
    format="%d-%m-%Y"
)


In [55]:
orders_by_month = (
    restaurant_merge
    .groupby(restaurant_merge["order_date"].dt.to_period("M"))
    .size()
)

orders_by_month


order_date
2023-01    804
2023-02    785
2023-03    903
2023-04    812
2023-05    844
2023-06    784
2023-07    859
2023-08    851
2023-09    812
2023-10    863
2023-11    807
2023-12    849
2024-01     27
Freq: M, dtype: int64

In [51]:
restaurant_merge.head()


,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,2023-02-18,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


2.User behavior patterns

In [63]:
orders_per_user = (
    restaurant_merge
    .groupby("user_id")
    .size()
    .sort_values(ascending=False)
)
orders_per_user.head()


user_id
2973    13
1515    12
874     11
1337    11
496     11
dtype: int64

In [64]:
avg_spend_user = (
    restaurant_merge
    .groupby("user_id")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

avg_spend_user.head()


user_id
2429    1497.42
889     1492.63
1843    1484.24
1882    1481.84
925     1476.18
Name: total_amount, dtype: float64

3.City-wise performance

In [54]:
orders_by_city = (
    restaurant_merge
    .groupby("city")
    .size()
    .sort_values(ascending=False)
)

orders_by_city


city
Bangalore    2751
Chennai      2469
Pune         2430
Hyderabad    2350
dtype: int64

In [56]:
revenue_by_city = (
    restaurant_merge
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

revenue_by_city


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

Cuisine-wise performance

In [57]:
orders_by_cuisine = (
    restaurant_merge
    .groupby("cuisine")
    .size()
    .sort_values(ascending=False)
)
orders_by_cuisine

cuisine
Mexican    2581
Italian    2532
Indian     2469
Chinese    2418
dtype: int64

In [58]:
revenue_by_cuisine = (
    restaurant_merge
    .groupby("cuisine")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
revenue_by_cuisine

cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

4.Membership impact (Gold vs Regular)

In [59]:
orders_by_membership = (
    restaurant_merge
    .groupby("membership")
    .size()
)
orders_by_membership

membership
Gold       4987
Regular    5013
dtype: int64

In [60]:
avg_order_value_membership = (
    restaurant_merge
    .groupby("membership")["total_amount"]
    .mean()
)
avg_order_value_membership

membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

5.Revenue distribution and seasonality

In [61]:
monthly_revenue = (
    restaurant_merge
    .groupby(restaurant_merge["order_date"].dt.to_period("M"))["total_amount"]
    .sum()
)
monthly_revenue

order_date
2023-01    645818.76
2023-02    630867.40
2023-03    716738.98
2023-04    666097.18
2023-05    668428.61
2023-06    610822.93
2023-07    688559.45
2023-08    694987.58
2023-09    653838.07
2023-10    699187.73
2023-11    637772.43
2023-12    681303.50
2024-01     17201.50
Freq: M, Name: total_amount, dtype: float64

In [62]:
restaurant_merge["total_amount"].describe()

count    10000.000000
mean       801.162412
std        405.458753
min        100.200000
25%        446.310000
50%        806.295000
75%       1149.227500
max       1499.830000
Name: total_amount, dtype: float64

MCQS
1.

In [66]:
gold_orders = restaurant_merge[restaurant_merge["membership"] == "Gold"]
gold_city_revenue = (
    gold_orders
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


In [67]:
gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [68]:
gold_city_revenue.head(1)


city
Chennai    1080909.79
Name: total_amount, dtype: float64

2.

In [69]:
avg_order_value_by_cuisine = (
    restaurant_merge
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
avg_order_value_by_cuisine


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [70]:
avg_order_value_by_cuisine.idxmax(), avg_order_value_by_cuisine.max()


('Mexican', 808.0213444401395)

3.

In [71]:
user_total_spend = (
    restaurant_merge
    .groupby("user_id")["total_amount"]
    .sum()
)
high_value_users = user_total_spend[user_total_spend > 1000]
high_value_users.count()

np.int64(2544)

In [72]:
high_value_users.head()


user_id
1    1289.66
2    7564.12
3    1839.51
4    3741.16
5    5742.88
Name: total_amount, dtype: float64

4

In [94]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

restaurant_merge['rating_range'] = pd.cut(restaurant_merge['rating'], bins=bins, labels=labels)

rating_rev = restaurant_merge.groupby('rating_range', observed=False)['total_amount']\
.sum().sort_values(ascending=False)

print("Rating range with highest total revenue:")
rating_rev.head(1)

Rating range with highest total revenue:


rating_range
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

5

In [79]:
gold_city_avg = (
    restaurant_merge[restaurant_merge["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_avg

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

6

In [80]:
restaurants_per_cuisine = (
    restaurant_merge
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)
revenue_per_cuisine = (
    restaurant_merge
    .groupby("cuisine")["total_amount"]
    .sum()
)
cuisine_summary = pd.DataFrame({
    "restaurant_count": restaurants_per_cuisine,
    "total_revenue": revenue_per_cuisine
}).sort_values("restaurant_count")

cuisine_summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


7

In [81]:
total_orders = len(restaurant_merge)
gold_orders = len(restaurant_merge[restaurant_merge["membership"] == "Gold"])

percentage_gold = round((gold_orders / total_orders) * 100)
percentage_gold


50

8

In [111]:
rest_stats = restaurant_merge.groupby('restaurant_name')\
.agg(avg_value=('total_amount','mean'), total_orders=('order_id','count'))
high_avg_rest = rest_stats[rest_stats['total_orders']<20].sort_values('avg_value', ascending=False)
print(" Restaurant with highest avg order value but <20 orders:")
print(high_avg_rest.head(1))

 Restaurant with highest avg order value but <20 orders:
                   avg_value  total_orders
restaurant_name                           
Restaurant_294   1040.222308            13


9

In [84]:
restaurant_merge \
    .groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [104]:
combo_rev = restaurant_merge.groupby(['membership','cuisine'])['total_amount'].sum()

mcq_options = [
    ('Gold','Indian'),
    ('Gold','Italian'),
    ('Regular','Indian'),
    ('Regular','Chinese')
]

filtered_combo = combo_rev.loc[mcq_options]\
.sort_values(ascending=False)

print(" Correct Answer from options:")
filtered_combo.head(1)

 Correct Answer from options:


membership  cuisine
Gold        Italian    1005779.05
Name: total_amount, dtype: float64

10

In [85]:
restaurant_merge["quarter"] = restaurant_merge["order_date"].dt.to_period("Q")

quarter_revenue = (
    restaurant_merge
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

11

In [86]:
gold_orders_count = len(
    restaurant_merge[restaurant_merge["membership"] == "Gold"]
)
gold_orders_count

4987

12

In [87]:
hyderabad_revenue = (
    restaurant_merge[restaurant_merge["city"] == "Hyderabad"]["total_amount"]
    .sum()
)
round(hyderabad_revenue)

1889367

13

In [88]:
distinct_users = restaurant_merge["user_id"].nunique()
distinct_users

2883

14

In [89]:
avg_order_value_gold = (
    restaurant_merge[restaurant_merge["membership"] == "Gold"]["total_amount"]
    .mean()
)
round(avg_order_value_gold, 2)

np.float64(797.15)

15

In [90]:
high_rating_orders = len(
    restaurant_merge[restaurant_merge["rating"] >= 4.5]
)
high_rating_orders

3374

16

In [91]:
top_gold_city = (
    restaurant_merge[restaurant_merge["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)
top_gold_city
orders_top_gold_city = len(
    restaurant_merge[
        (restaurant_merge["membership"] == "Gold") &
        (restaurant_merge["city"] == top_gold_city)
    ]
)
orders_top_gold_city

1337